In [6]:
import pandas as pd

In [7]:
path_to_file = '2_taxi_nyc.csv'
taxi = pd.read_csv(path_to_file)
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


## 2.3
Проверьте, сколько всего строк и столбцов имеется в датасете.

In [3]:
taxi.shape

(29101, 14)

## 2.4
Давайте посмотрим на типы колонок. Все ли из них считались правильно? В качестве ответа выберите тип, преобладающий в датасете.

In [4]:
taxi.dtypes.value_counts()

float64    9
object     4
int64      1
dtype: int64

## 2.5
В названиях столбцов pcp 01, pcp 06, pcp 24 встречается пробел. Это не очень удобно, если в дальнейшем вы планируете обращаться к колонкам через точку, без использования кавычек и скобочек.  
Замените пробел в названиях на знак нижнего подчеркивания.  
Данные сохранены в переменную taxi. Данные должны быть изменены в этом же датафрейме.

In [13]:
# taxi.rename(
#     columns={'pcp 01': 'pcp_01', 'pcp 06': 'pcp_06', 'pcp 24': 'pcp_24'}, 
#     inplace=True
# )

In [5]:
taxi.columns = taxi.columns.str.replace(' ', '_')

## 2.6
Вас попросили узнать, сколько записей (строк) в датафрейме относятся к району Манхэттен (Manhattan). Получить ответ на данный вопрос можно было бы, например, с помощью следующей команды:  
taxi.query("borough == 'Manhattan'").shape[0]  
А если мы хотим посмотреть, сколько раз встречается каждый из районов? Неужели придется использовать подобную конструкцию для каждого уровня переменной? Здесь на помощь приходит метод value_counts(), подробнее о котором можно почитать в конспекте!  
А теперь вопрос: сколько раз в данных встречается район Бруклин (Brooklyn)?

In [11]:
taxi.borough.value_counts(dropna=False)

Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
NaN              3043
Name: borough, dtype: int64

## 2.7
Следующая задача: выяснить, из какого района было совершено наибольшее количество поездок за весь период.  
Для начала посчитайте общее количество поездок (pickups), без группировки.

In [12]:
taxi.pickups.sum()

14265773

## 2.8
А теперь сгруппируйте данные по району (borough) и укажите, из какого пункта было совершено больше всего поездок.

In [26]:
taxi.groupby('borough').pickups.sum().sort_values(ascending=False)

borough
Manhattan        10367841
Brooklyn          2321035
Queens            1343528
Bronx              220047
Staten Island        6957
EWR                   105
Name: pickups, dtype: int64

In [22]:
taxi.groupby('borough').pickups.sum().sort_values(ascending=False).idxmax()

'Manhattan'

## 2.9
Задание со звёздочкой!  
Еще несколько полезных методов в pandas! Возможно, в предыдущем степе вы просто посмотрели на данные или отсортировали значения. Ускорить данный процесс в дальнейшем помогут методы idxmin() и idxmax(), которые возвращают индекс минимального или максимального значения.  
- idxmin – индекс минимального значения
- idxmax – индекс максимального значения  

Сохраните название района с наименьшим числом поездок в переменную min_pickups, применив подходящий метод.  
Исходные данные сохранены в переменную taxi.

In [24]:
min_pickups = taxi.groupby('borough').pickups.sum().sort_values(ascending=False).idxmin()

In [25]:
min_pickups

'EWR'

## 2.10
Продолжим изучение данных и посмотрим на число поездок в выходные дни. Сгруппируйте данные по двум признакам: району города и является ли день выходным (колонки borough и hday). Сравните среднее число поездок, и выберите районы, из которых по праздникам в среднем поступает больше заказов, чем в обычные дни. 

In [29]:
taxi.groupby(['borough', 'hday'], dropna=False).pickups.mean()

borough        hday
Bronx          N         50.771073
               Y         48.065868
Brooklyn       N        534.727969
               Y        527.011976
EWR            N          0.023467
               Y          0.041916
Manhattan      N       2401.302921
               Y       2035.928144
Queens         N        308.899904
               Y        320.730539
Staten Island  N          1.606082
               Y          1.497006
NaN            N          2.057456
               Y          2.050420
Name: pickups, dtype: float64

In [36]:
pd.merge(
    taxi[taxi.hday == 'N'].groupby('borough', dropna=False).pickups.mean(),
    taxi[taxi.hday == 'Y'].groupby('borough', dropna=False).pickups.mean(),
    left_index=True,
    right_index=True
)

,pickups_x,pickups_y
borough,,
Bronx,50.771073,48.065868
Brooklyn,534.727969,527.011976
EWR,0.023467,0.041916
Manhattan,2401.302921,2035.928144
Queens,308.899904,320.730539
Staten Island,1.606082,1.497006
NaN,2.057456,2.050420


In [38]:
taxi[taxi.hday == 'N'].groupby('borough', dropna=False).pickups.mean() < taxi[taxi.hday == 'Y'].groupby('borough', dropna=False).pickups.mean()

borough
Bronx            False
Brooklyn         False
EWR               True
Manhattan        False
Queens            True
Staten Island    False
NaN              False
Name: pickups, dtype: bool

## 2.11
Для каждого района посчитайте число поездок по месяцам. Отсортируйте полученные значения по убыванию и сохраните результирующий датафрейм в pickups_by_mon_bor.  
Обратите внимание, что итоговый датасет должен состоять из 3-х колонок - pickup_month, borough, pickups.  
Данные сохранены в переменную taxi.

In [22]:
pickups_by_mon_bor = taxi\
                    .groupby(['pickup_month', 'borough'], as_index=False)\
                    .pickups\
                    .sum()\
                    .sort_values('pickups', ascending=False)

In [23]:
pickups_by_mon_bor.head()

,pickup_month,borough,pickups
21,Jun,Manhattan,1995388
33,May,Manhattan,1888800
9,Feb,Manhattan,1718571
27,Mar,Manhattan,1661261
3,Apr,Manhattan,1648278


## 2.12
Теперь задача посложнее! Попрактикуемся в написании собственных функций и их применении к датафрейму. Поскольку данные о поездках в Нью-Йорке, температура представлена в градусах Фаренгейта. 

Напишите функцию temp_to_celcius, которая получает на вход колонку с температурой в °F и возвращает значения, переведенные в градусы Цельсия.

Формула:

Celsius = (Fahrenheit − 32) * 5.0 / 9.0

In [11]:
def temp_to_celcius(Fahrenheits):
    return Fahrenheits.apply(lambda Fahrenheit: (Fahrenheit - 32) * 5.0 / 9.0)

In [12]:
temp_to_celcius(taxi.temp)

0        -1.111111
1        -1.111111
2        -1.111111
3        -1.111111
4        -1.111111
           ...    
29096    23.888889
29097    23.888889
29098    23.888889
29099    23.888889
29100    23.888889
Name: temp, Length: 29101, dtype: float64

In [21]:
def temp_to_celcius(Fahrenheits):
    return (Fahrenheits - 32) * 5.0 / 9.0

In [22]:
temp_to_celcius(taxi.temp)

0        -1.111111
1        -1.111111
2        -1.111111
3        -1.111111
4        -1.111111
           ...    
29096    23.888889
29097    23.888889
29098    23.888889
29099    23.888889
29100    23.888889
Name: temp, Length: 29101, dtype: float64

In [21]:
taxi.head(5)

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


## 3.1
Строка names содержит имена в формате 'Name1 Surname1, Name2 Surname2, …'.  
Поместите в переменную names_list список со строками имён – ['Name1 Surname1', 'Name2 Surname2', …]
Строка names уже задана.

In [5]:
names = 'Name1 Surname1, Name2 Surname2, …'
names_list = names.split(', ')
names_list

['Name1 Surname1', 'Name2 Surname2', '…']

## 3.2
В переменной df содержатся данные. Отберите из них только те, где wealth принимает значение 'medium', а возраст составляет больше 35. Поместите результат в переменную medium_35.

In [23]:
taxi[(taxi.borough == 'Queens') & (taxi.pickups > 800)]

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
27640,2015-06-22 00:00:00,Jun,Queens,831,N,5.0,10.0,80.0,67.0,1007.8,0.0,0.01,0.0,0.0


In [ ]:
medium_35 = df[(df.wealth == 'medium') & (df.age > 35)]

## 3.3
Ваш коллега-новичок перепутал названия колонок в таблице df из предыдущих заданий и прислал вам такой вариант:
Воспользуйтесь методом rename и приведите таблицу к более каноничному виду, сохранив её в ту же самую переменную df.

In [28]:
taxi.rename(columns={'pickup_month': 'pm'}).head(1)

,pickup_dt,pm,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


In [26]:
df.rename(columns={'Клиент id': 'client_id', 'Пол': 'sex' ,'Благосостояние': 'wealth', 'Возраст': 'age'}, inplace=True)

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29096,2015-06-30 23:00:00,Jun,EWR,0,N,7.0,10.0,75.0,65.0,1011.8,0.0,0.0,0.0,0.0
29097,2015-06-30 23:00:00,Jun,Manhattan,3828,N,7.0,10.0,75.0,65.0,1011.8,0.0,0.0,0.0,0.0
29098,2015-06-30 23:00:00,Jun,Queens,580,N,7.0,10.0,75.0,65.0,1011.8,0.0,0.0,0.0,0.0
29099,2015-06-30 23:00:00,Jun,Staten Island,0,N,7.0,10.0,75.0,65.0,1011.8,0.0,0.0,0.0,0.0


In [8]:
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
